# PATH ridership over time

Cleaned + Plotted Port Authority data from https://www.panynj.gov/path/en/about/stats.html

In [1]:
from utz import *
from utz.colors import colors_lengthen
from datetime import timedelta
from ire import export
from path_data.paths import DATA, ALL_PQT

Papermill parameters:

In [2]:
force = True
default_show = 'png'
W = 1200
H = 600
end_month = None

In [3]:
rename_stations = {
    '9thStreet': '9th Street',
    '14thStreet': '14th Street',
    '23rdStreet': '23rd Street',
    '33rdStreet': '33rd Street',
    'Pavonia/ Newport': 'Newport',
}

In [4]:
df = concat([
    read_parquet(pqt_path)
    for pqt_path in sorted(glob.glob(f'{DATA}/2*.pqt'))
    if fullmatch(r'\d{4}\.pqt', basename(pqt_path))
]).reset_index()
stations = df.station.apply(lambda s: rename_stations.get(s, s))
df['station'] = stations
df = df[~df.station.str.contains('TOTAL')].copy()
df['dt'] = to_dt(df.month)
df['year'] = df.dt.dt.year
df['month_idx'] = df.dt.dt.month
df

,month,station,avg daily,avg weekday,avg sat,avg sun,avg holiday,total,total weekday,total sat,total sun,total holiday,dt,year,month_idx
0,2012-01-01,Christopher Street,3619,4256,2708,2433,2040,112187,85113,10830,12163,4081,2012-01-01,2012,1
1,2012-01-01,9th Street,4087,4483,3695,3436,2533,126682,89658,14780,17178,5066,2012-01-01,2012,1
2,2012-01-01,14th Street,7370,8768,5346,4730,4041,228483,175369,21384,23648,8082,2012-01-01,2012,1
3,2012-01-01,23rd Street,6374,8345,2924,2563,3096,197597,166895,11698,12813,6191,2012-01-01,2012,1
4,2012-01-01,33rd Street,28586,34797,17350,16716,18627,886167,695937,69398,83578,37254,2012-01-01,2012,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2628,2024-11-01 00:00:00,Journal Square,20096,23891,15555,11848,15891,602869,430030,77775,47390,47674,2024-11-01,2024,11
2629,2024-11-01 00:00:00,Grove Street,14304,17143,11584,8594,9416,429112,308568,57919,34377,28248,2024-11-01,2024,11
2630,2024-11-01 00:00:00,Exchange Place,9531,11848,6821,4931,6277,285915,213255,34106,19724,18830,2024-11-01,2024,11
2631,2024-11-01 00:00:00,Newport,9941,12572,6352,4567,7298,298216,226296,31759,18266,21895,2024-11-01,2024,11


In [5]:
dfs = []
for pqt_path in sorted(glob.glob(f'{DATA}/2*-day-types.pqt')):
    year = fullmatch(r'(?P<y>\d{4})-day-types\.pqt', basename(pqt_path))['y']
    dfs.append(read_parquet(pqt_path).assign(year=int(year)))
day_hists = pd.concat(dfs).reset_index().set_index(['year', 'month'])
day_hists

weekdays  saturdays  sundays  holidays
year month                                        
2012 1            20          4        5         2
     2            20          4        4         1
     3            22          5        4         0
     4            21          4        5         0
     5            22          4        4         1
...              ...        ...      ...       ...
2024 7            22          4        4         1
     8            22          5        4         0
     9            20          4        5         1
     10           22          4        4         1
     11           18          5        4         3

[155 rows x 4 columns]

In [21]:
m = (
    df.merge(
        day_hists,
        how='left',
        left_on=['year', 'month_idx'],
        right_index=True,
    )
    .rename(columns={
        'saturdays': 'sats',
        'sundays': 'suns',
    })
    .drop(columns=['year', 'month_idx', 'dt', 'avg daily', 'total'])
)

for k in ['weekday', 'sat', 'sun', 'holiday']:
    tk = f'total {k}'
    ak = f'avg {k}'
    nk = f'{k}s'
    avg = m[tk] / m[nk]
    mask = (abs(avg - m[ak]) < .51) | ((m[nk] == 0) & (m[tk] == 0))
    if not all(mask):
        err(k)
        raise ValueError(m.loc[~mask, [tk, nk, ak]])
    m[ak] = avg

m['weekends'] = m.sats + m.suns
m['total weekend'] = m['total sat'] + m['total sun']
m['avg weekend'] = m['total weekend'] / m['weekends']

cols = ['month', 'station'] + [
    p+k+s
    for p, s in [('total ', ''), ('avg ', ''), ('', 's')]
    for k in ['weekday', 'weekend', 'sat', 'sun', 'holiday']
]
cols += [ c for c in m if c not in cols ]
m = m[cols]
m['month'] = m.month.astype('datetime64[us]')
for c in m:
    if m[c].dtype == np.int64:
        m[c] = m[c].astype('int32')
m

,month,station,total weekday,total weekend,total sat,total sun,total holiday,avg weekday,avg weekend,avg sat,avg sun,avg holiday,weekdays,weekends,sats,suns,holidays
0,2012-01-01,Christopher Street,85113,22993,10830,12163,4081,4255.650000,2554.777778,2707.5,2432.60,2040.500000,20,9,4,5,2
1,2012-01-01,9th Street,89658,31958,14780,17178,5066,4482.900000,3550.888889,3695.0,3435.60,2533.000000,20,9,4,5,2
2,2012-01-01,14th Street,175369,45032,21384,23648,8082,8768.450000,5003.555556,5346.0,4729.60,4041.000000,20,9,4,5,2
3,2012-01-01,23rd Street,166895,24511,11698,12813,6191,8344.750000,2723.444444,2924.5,2562.60,3095.500000,20,9,4,5,2
4,2012-01-01,33rd Street,695937,152976,69398,83578,37254,34796.850000,16997.333333,17349.5,16715.60,18627.000000,20,9,4,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2628,2024-11-01,Journal Square,430030,125165,77775,47390,47674,23890.555556,13907.222222,15555.0,11847.50,15891.333333,18,9,5,4,3
2629,2024-11-01,Grove Street,308568,92296,57919,34377,28248,17142.666667,10255.111111,11583.8,8594.25,9416.000000,18,9,5,4,3
2630,2024-11-01,Exchange Place,213255,53830,34106,19724,18830,11847.500000,5981.111111,6821.2,4931.00,6276.666667,18,9,5,4,3
2631,2024-11-01,Newport,226296,50025,31759,18266,21895,12572.000000,5558.333333,6351.8,4566.50,7298.333333,18,9,5,4,3


In [22]:
m.dtypes

month            datetime64[us]
station                  object
total weekday             int32
total weekend             int32
total sat                 int32
total sun                 int32
total holiday             int32
avg weekday             float64
avg weekend             float64
avg sat                 float64
avg sun                 float64
avg holiday             float64
weekdays                  int32
weekends                  int32
sats                      int32
suns                      int32
holidays                  int32
dtype: object

In [23]:
if force or not exists(ALL_PQT):
    m.to_parquet(ALL_PQT, index=False, engine='fastparquet')

In [ ]:
station_hist = stations.value_counts()
station_hist

In [ ]:
assert len(station_hist.value_counts()) == 1

In [ ]:
if not default_show:
    default_show = None

if end_month is None:
    end_month = (df.month.max() + pd.Timedelta('32d')).strftime('%Y-%m')
end_month

In [ ]:
mt = m.drop(columns='station').groupby('month').sum()
month_dt = to_dt(mt.index.to_series())
months = month_dt.dt
month_idxs = months.month.rename('month_idx')
years = months.year.rename('year')
mt = sxs(years, month_idxs, mt)
mt

In [ ]:
from calendar import month_abbr
pivoted = (
    mt
    .assign(month=mt.month_idx.apply(lambda m: month_abbr[m]))
    .set_index(['month_idx', 'month'])
    [['year', 'avg weekday', 'avg weekend', 'avg sat', 'avg sun']]
    .pivot(columns='year')
    .replace(nan, 0).astype(int)
    .sort_index()
    .reset_index(level=0, drop=True)
)
pivoted

In [ ]:
import plotly.express as px
from IPython.display import Image

In [ ]:
gridcolor = '#ccc'

def default_plot(fig, hoverformat=','):
    return (
        fig
        .update_layout(
            title_x=0.5,
            paper_bgcolor='white',
            plot_bgcolor='white',
            legend=dict(traceorder='reversed'),
            hovermode='x',
        ).update_xaxes(
            tickangle=-45,
            gridcolor=gridcolor,
        ).update_traces(
            hovertemplate=None,
        ).update_yaxes(
            gridcolor=gridcolor,
            hoverformat=hoverformat,
        )
    )

In [ ]:
def stations_stack(
    y, title, name=None,
    start=None, end=None,
    dtick=None, tickangle=-45,
    width=W, height=H,
    show=default_show,
):
    if isinstance(start, str):
        start = to_dt(start)
    start = start or to_dt('2012')
    start -= timedelta(days=15)
    if isinstance(end, str):
        end = to_dt(end)
    end = end or to_dt(end_month)
    end -= timedelta(days=15)
    dims = dict(width=width, height=height)
    fig = default_plot(
        px.bar(
            m.reset_index(),
            x='month', y=y, color='station',
            title=title,
            labels={
                'station': 'Station',
                y: title,
                'month': '',
            }
        )
    ).update_xaxes(
        range=[start, end],
        dtick=dtick,
    ).update_layout(**dims)
    if name:
        path = f'img/{name}.png'
        print(f'Saving {path}')
        fig.write_image(path, **dims)
    return export(fig, name, show=show)

In [ ]:
num_years = len(mt.year.unique())
num_years

In [ ]:
px_colors = px.colors.sequential.Inferno
colors = list(reversed(colors_lengthen(px_colors, num_years)))
colors

In [ ]:
month_names = [ to_dt('2022-%02d' % i).strftime('%b') for i in range(1, 13) ]
print(' '.join(month_names))

In [ ]:
def grouped_month_plot(y, title, width=W, height=H, show=default_show):
    fig = px.bar(
        mt,
        x='month_idx', y=y,
        color=mt.year.astype(str),
        color_discrete_sequence=colors,
        labels={
            'color': 'Year',
            'month_idx': '',
            y: title,
        },
        barmode='group',
    )
    dims = dict(width=width, height=height)
    fig = fig.update_layout(
        title=f'{title} (grouped by month)',
        title_x=0.5,
        xaxis=dict(
            tickmode = 'array',
            tickvals = list(range(1, 13)),
            ticktext = month_names,
        ),
        **dims,
    )
    name = f'{y}_month_grouped.png'
    path = f'img/{name}'
    fig.write_image(path, **dims)
    return export(fig, name, show=show)

# Weekdays

In [ ]:
stations_stack(
    'avg weekday',
    'Average weekday PATH ridership',
    dtick="M12",
    name='weekdays',
)

In [ ]:
stations_stack(
    'avg weekday',
    'Average weekday PATH ridership (2020-Present)',
    name='weekdays_2020:',
    dtick='M3',
    start='2020',
)

In [ ]:
grouped_month_plot('avg weekday', 'Average Weekday Rides')

# Weekends

In [ ]:
stations_stack(
    'avg weekend',
    'Average Weekend PATH ridership',
    dtick="M12",
    name='weekends',
)

In [ ]:
stations_stack(
    'avg weekend',
    'Average Weekend PATH ridership (2020-Present)',
    dtick="M3",
    name='weekends_2020:',
    start='2020',
)

In [ ]:
grouped_month_plot('avg weekend', 'Average Weekend Rides')

# Combined

## Average Daily PATH Ridership <a id="avg-daily"></a>

In [ ]:
def lines(
    df, name, xname, y_fmt,
    hoverx=True, dir='img',
    legend_lr=False,
    ann_offset=10,
    xtick=None, xtickformat=None,
    ytick=None, ytickformat=None,
    show=default_show,
    w=1000, h=600,
    **kwargs,
):
    fig = px.line(
        df,
        labels={
            'variable': '',
            'value': xname,
            'month': '',
        }
    )
    idx = df.index.to_series()
    for k in df:
        x = idx.iloc[-1]
        y = df[k].iloc[-1]
        x_str = x.strftime("%b '%y")
        y_str = format(y, y_fmt)
        fig.add_annotation(
            x=x,
            ax=idx.iloc[-ann_offset],
            axref="x",
            y=y,
            text=f'{x_str}: {y_str}',
        )
    fig = plots.save(
        fig,
        name=name,
        x=dict(dtick=xtick, tickformat=xtickformat),
        y=dict(dtick=ytick, tickformat=ytickformat),
        legend=(
            dict(
                yanchor="bottom",
                y=0.03,
                xanchor="right",
                x=0.99,
            ) if legend_lr else dict(
                yanchor="top",
                y=0.99,
                xanchor="right",
                x=0.99,
            )
        ),
        hoverx=hoverx,
        dir=dir,
        w=w, h=h,
        **kwargs,
    )
    return export(fig, name, show=show)

In [ ]:
lines(
    mt[['avg weekday', 'avg weekend',]].rename(columns={
        'avg weekday': 'Avg Weekday',
        'avg weekend': 'Avg Weekend',
    }),
    name='avg_day_types',
    xname = 'Daily Rides',
    xtick = "M12",
    hovertemplate='%{y:,.0f}',
    y_fmt = ',.0f',
    legend_lr=False,
)

## 2020-2022 Ridership vs. 2019

In [ ]:
mt20 = mt[month_dt >= to_dt('2020')]
mt19 = mt[mt.year == 2019]
mt19s = pd.concat([ mt19 for i in range((len(mt20) + len(mt19) - 1) // len(mt19)) ]).iloc[:len(mt20)]

keys = ['avg weekday', 'avg weekend']
cmp19 = (
    sxs(
        mt19s.reset_index(drop=True)[keys].rename(columns={ key: f'{key} 2019' for key in keys }),
        mt20.reset_index()[keys + ['month']],
    )
    .set_index('month')
)
for k in keys:
    cmp19[f'{k} frac'] = cmp19[k] / cmp19[f'{k} 2019']
cmp19

## PATH Ridership: 2020-2023 (YTD) vs. 2019 <a id="vs-2019"></a>

In [ ]:
lines(
    cmp19[[f'{k} frac' for k in keys]].rename(columns={
        'avg weekday frac': 'Avg Weekday (% of 2019)',
        'avg weekend frac': 'Avg Weekend (% of 2019)',
    }),
    name='vs_2019',
    xname='% of 2019 ridership',
    xtick="M3",
    ytickformat=".0%",
    ann_offset=3,
    hovertemplate='%{y:.1%}',
    y_fmt='.1%',
    legend_lr=True,
)

# {Month,Station} data

In [ ]:
export(df.set_index(['month', 'station']), 'path', fmts={'month': "%b '%y"}, per_page=20);